##Dataset link

https://drive.google.com/drive/folders/1dVUSjeGoFbAxQ892-z6wtRyNEeyPIPry?usp=sharing

In [1]:
import tensorflow as tf
from tensorflow.python.client import device_lib

print("Number GPUs Avalable: ", 
      len(tf.config.experimental.list_physical_devices('GPU')))
print(tf.config.experimental.list_physical_devices('GPU'))
print(device_lib.list_local_devices())

Number GPUs Avalable:  0
[]
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6455001660142142209
xla_global_id: -1
]


In [2]:
# import os
# os.mkdir('datasets')
# os.mkdir('datasets/signature/')
# os.mkdir('datasets/fake/')


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
width = 224
height = 224
batchSize = 32

In [5]:
data_dir = '/content/gdrive/MyDrive/Comviz_3/datasets'

In [6]:
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [7]:
from tensorflow.keras.preprocessing import image_dataset_from_directory

train_ds = image_dataset_from_directory(
    data_dir+"/train",
    validation_split = 0.2,
    subset = 'training',
    seed = 12,
    image_size = (width, height),
    batch_size = batchSize,
    label_mode = 'categorical'
)

validation_ds = image_dataset_from_directory(
    data_dir+"/train",
    validation_split = 0.2,
    subset = 'validation',
    seed = 12,
    image_size = (width, height),
    batch_size = batchSize,
    label_mode = 'categorical'
)

Found 354 files belonging to 2 classes.
Using 284 files for training.
Found 354 files belonging to 2 classes.
Using 70 files for validation.


In [8]:
classNames = train_ds.class_names
print(classNames)

['fake', 'real']


#Building Model

In [9]:
from tensorflow.keras.layers import Input, Conv2D, Activation, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.metrics import Recall, Precision

def AlexnetModel(input_shape, num_classes):

  # Input Layer
  input = Input(shape = input_shape)

  # Layer 1
  x = Conv2D(96, (11, 11), strides = 4, padding='same')(input)
  x = Activation('relu')(x)
  x = MaxPooling2D((3,3), strides = 2)(x)

  # layer 2
  x = Conv2D(256, (5, 5), strides = 2, padding='same')(x)
  x = Activation('relu')(x)
  x = MaxPooling2D((3,3), strides = 2)(x)

  # layer 3
  x = Conv2D(384, (3, 3), strides = 1, padding='same')(x)
  x = Activation('relu')(x)

  # layer 4
  x = Conv2D(384, (3, 3), strides = 1, padding='same')(x)
  x = Activation('relu')(x)

  # layer 5
  x = Conv2D(256, (3, 3), strides = 1, padding='same')(x)
  x = Activation('relu')(x)
  x = MaxPooling2D((3,3), strides = 2)(x)

  # Flatten
  x = Flatten()(x)

  # Fully Connected Layer 1
  x = Dense(4096)(x)
  x = Activation('relu')(x)
  x = Dropout(0.5)(x)

  # Fully Connected Layer 2
  x = Dense(4096)(x)
  x = Activation('relu')(x)
  x = Dropout(0.5)(x)

  # Output Layer
  x = Dense(num_classes, activation = 'softmax')(x)

  # Compile the model
  model = Model(inputs = input, outputs = x)
  model.compile(
      optimizer = 'adam',
      loss = 'categorical_crossentropy',
      metrics = ['accuracy', Recall(), Precision()]
  )

  return model

In [10]:
myModel = AlexnetModel(input_shape = [width, height, 3], num_classes = len(classNames))
myModel.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 56, 56, 96)        34944     
                                                                 
 activation (Activation)     (None, 56, 56, 96)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 27, 27, 96)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 256)       614656    
                                                                 
 activation_1 (Activation)   (None, 14, 14, 256)       0         
                                                             

#Train Model

In [12]:
from datetime import datetime

start_time = datetime.now()

history = myModel.fit(
    train_ds,
    batch_size = batchSize,
    epochs = 10,
    validation_data = validation_ds,
    verbose = 1
)

end_time = datetime.now()
print("Duration: "+str(end_time - start_time))
myModel.save('/content/modelCNN_Alexnet')

Epoch 1/10
9/9 [==============================] - 34s 4s/step - loss: 77.3892 - accuracy: 0.5599 - recall: 0.5599 - precision: 0.5599 - val_loss: 0.6077 - val_accuracy: 0.7143 - val_recall: 0.7143 - val_precision: 0.7143
Epoch 2/10
9/9 [==============================] - 24s 3s/step - loss: 0.5833 - accuracy: 0.7077 - recall: 0.7077 - precision: 0.7077 - val_loss: 0.5774 - val_accuracy: 0.7286 - val_recall: 0.7286 - val_precision: 0.7286
Epoch 3/10
9/9 [==============================] - 22s 2s/step - loss: 0.5035 - accuracy: 0.7958 - recall: 0.7958 - precision: 0.7958 - val_loss: 0.4225 - val_accuracy: 0.8286 - val_recall: 0.8286 - val_precision: 0.8286
Epoch 4/10
9/9 [==============================] - 21s 2s/step - loss: 0.3526 - accuracy: 0.8732 - recall: 0.8732 - precision: 0.8732 - val_loss: 0.2247 - val_accuracy: 0.9286 - val_recall: 0.9286 - val_precision: 0.9286
Epoch 5/10
9/9 [==============================] - 24s 2s/step - loss: 0.1897 - accuracy: 0.9296 - recall: 0.9296 - prec

In [13]:
# Average
import numpy as np

print("Duration: "+str(end_time - start_time))
print("\nTesting Evaluation")
print("Average loss: ", np.average(history.history['loss']))
print("Average accuracy: ", np.average(history.history['accuracy']))
print("Average recall: ", np.average(history.history['recall']))
print("Average precision: ", np.average(history.history['precision']))

print("\nValidation Evaluation")
print("Average loss: ", np.average(history.history['val_loss']))
print("Average accuracy: ", np.average(history.history['val_accuracy']))
print("Average recall: ", np.average(history.history['val_recall']))
print("Average precision: ", np.average(history.history['val_precision']))

Duration: 0:05:20.283081

Testing Evaluation
Average loss:  7.971509512513876
Average accuracy:  0.8633802890777588
Average recall:  0.8633802890777588
Average precision:  0.8633802890777588

Validation Evaluation
Average loss:  0.31650623977184295
Average accuracy:  0.879999989271164
Average recall:  0.879999989271164
Average precision:  0.879999989271164


#Testing Model


In [15]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.math import confusion_matrix

test_data = ImageDataGenerator()
test_ds = test_data.flow_from_directory(
    '/content/gdrive/MyDrive/Comviz_3/datasets',
    target_size=(width, height),
    batch_size = batchSize,
    class_mode = 'categorical',
    shuffle=False
)

class_names = []
for key, value in test_ds.class_indices.items():
  class_names.append(key)

predictions = np.array([])
labels = np.array([])

for x, y in test_ds:
  predictions = np.concatenate([predictions, np.argmax(myModel.predict(x), axis = -1)])
  labels = np.concatenate([labels, np.argmax(y, axis=-1)])

matrix = confusion_matrix(labels = labels, predictions = predictions).numpy()

df = pd.DataFrame(matrix, columns = class_names, index = class_names)
df

Found 424 images belonging to 2 classes.
1/1 [==============================] - 1s 804ms/step


KeyboardInterrupt: ignored

In [16]:
# Average
import numpy as np

print("Duration: "+str(end_time - start_time))
print("\nTesting Evaluation")
print("Average loss: ", np.average(history.history['loss']))
print("Average accuracy: ", np.average(history.history['accuracy']))
print("Average recall: ", np.average(history.history['recall']))
print("Average precision: ", np.average(history.history['precision']))

print("\nValidation Evaluation")
print("Average loss: ", np.average(history.history['val_loss']))
print("Average accuracy: ", np.average(history.history['val_accuracy']))
print("Average recall: ", np.average(history.history['val_recall']))
print("Average precision: ", np.average(history.history['val_precision']))

Duration: 0:05:20.283081

Testing Evaluation
Average loss:  7.971509512513876
Average accuracy:  0.8633802890777588
Average recall:  0.8633802890777588
Average precision:  0.8633802890777588

Validation Evaluation
Average loss:  0.31650623977184295
Average accuracy:  0.879999989271164
Average recall:  0.879999989271164
Average precision:  0.879999989271164
